In [1]:
import findspark
findspark.init()
findspark.find()

'/Users/glouno/anaconda3/lib/python3.11/site-packages/pyspark'

In [5]:
# Import PySpark
from pyspark.sql import SparkSession
# import the dataframe sql data types
from pyspark.sql.types import *

#Create SparkSession
spark = SparkSession.builder.appName('TPairport').getOrCreate()

#
# flightSchema describes the structure of the data in the flights.csv file
#
flightSchema = StructType([
  StructField("DayofMonth", IntegerType(), False),
  StructField("DayOfWeek", IntegerType(), False),
  StructField("Carrier", StringType(), False),
  StructField("OriginAirportID", IntegerType(), False),
  StructField("DestAirportID", IntegerType(), False),
  StructField("DepDelay", IntegerType(), False),
  StructField("ArrDelay", IntegerType(), False),
])
#
# Use the dataframe reader to read the file and 
#
flights = spark.read.csv('raw-flight-data.csv', schema=flightSchema, header=True)
flights.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 11:54:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/14 11:54:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/11/14 11:54:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [6]:
airports = spark.read.csv('airports.csv', header=True, inferSchema=True)
airports.show(10)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
|     10926|    Cordova|   AK|Merle K Mudhole S...|
|     14709|  Deadhorse|   AK|   Deadhorse Airport|
|     11336| Dillingham|   AK|  Dillingham Airport|
|     11630|  Fairbanks|   AK|Fairbanks Interna...|
|     11997|   Gustavus|   AK|    Gustavus Airport|
+----------+-----------+-----+--------------------+
only showing top 10 rows



In [7]:
airports.printSchema()

root
 |-- airport_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)



In [11]:
flights2 = spark.read.csv('flights.csv', schema=flightSchema, header=True)

#testing the differences:
diff_flights = flights.subtract(flights2)
diff_flights.show()
rows = diff_flights.count()
print(f"DataFrame Rows count : {rows}")

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|         1|        1|     UA|          14771|        12173|    null|    null|
|         3|        6|     AA|          12892|        12889|    null|    null|
|         5|        3|     OO|          14771|        10800|    null|    null|
|         6|        4|     MQ|          13930|        13487|    null|    null|
|         6|        6|     UA|          12892|        14771|    null|    null|
|         7|        7|     MQ|          13930|        11193|    null|    null|
|        10|        1|     UA|          11278|        12266|     223|    null|
|        10|        3|     AA|          12953|        11298|       8|    null|
|        11|        4|     UA|          12266|        11618|    null|    null|
|        12|        5|     AA|          13495|      

DataFrame Rows count : 23798


In [12]:
### exceptall() does not take out duplicates
diff_flights_wduplicates = flights.exceptAll(flights2)
diff_flights_wduplicates.show()
rows = diff_flights_wduplicates.count()
print(f"DataFrame Rows count : {rows}")

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        21|        7|     DL|          11278|        10397|       0|      10|
|        11|        4|     EV|          13198|        13244|    null|    null|
|        10|        3|     EV|          11618|        13487|    null|    null|
|        10|        3|     EV|          11618|        13487|    null|    null|
|        10|        3|     EV|          11618|        13487|    null|    null|
|        10|        3|     EV|          11618|        13487|    null|    null|
|        10|        3|     EV|          11618|        13487|    null|    null|
|        17|        3|     EV|          13930|        10693|    null|    null|
|         8|        1|     FL|          10397|        13204|      -1|     -14|
|         8|        1|     FL|          10397|      

DataFrame Rows count : 46224


In [13]:
flights.describe().show()

23/11/14 15:05:39 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+-------+------------------+------------------+-----------------+------------------+
|summary|       DayofMonth|         DayOfWeek|Carrier|   OriginAirportID|     DestAirportID|         DepDelay|          ArrDelay|
+-------+-----------------+------------------+-------+------------------+------------------+-----------------+------------------+
|  count|          2719418|           2719418|2719418|           2719418|           2719418|          2691974|           2690385|
|   mean|15.79747468024408|3.8983907586108497|   null| 12742.26441172339|12742.455345592329|10.53686662649788|  6.63768791455498|
| stddev|8.799860168985342|1.9859881390373295|   null|1501.9729397025724| 1501.969252892788|36.09952806643132|38.648814893900735|
|    min|                1|                 1|     9E|             10140|             10140|              -63|               -94|
|    max|               31|                 7|     YV|             15376|             1537

In [14]:
total_flights = flights.count()
unique_flights = flights.dropDuplicates().count()

print("Number of duplicate rows = ",total_flights - unique_flights)

Number of duplicate rows =  22435


In [15]:
total_flights = flights2.count()
unique_flights = flights2.dropDuplicates().count()

print("Number of duplicate rows for flights 2 = ",total_flights - unique_flights)

Number of duplicate rows for flights 2 =  5561


In [26]:
cities = airports.select("city", "name")    #remember, 'name' is airport name
cities.limit(5).show()

from pyspark.sql import functions as F

flightsByOrigin = flights\
.join(airports, flights.OriginAirportID == airports.airport_id)

flightsByOrigin.limit(5).show()

+-----------+--------------------+
|       city|                name|
+-----------+--------------------+
|Adak Island|                Adak|
|  Anchorage|Ted Stevens Ancho...|
|      Aniak|       Aniak Airport|
|     Barrow|Wiley Post/Will R...|
|     Bethel|      Bethel Airport|
+-----------+--------------------+

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
|        19|        5|  

In [28]:
flightsByOrigin.groupBy("city") \
    .agg(F.count(F.lit(1)).alias("Count"))\
    .orderBy("Count", ascending=False)\
    .show()

+-----------------+------+
|             city| Count|
+-----------------+------+
|          Chicago|177845|
|          Atlanta|149970|
|      Los Angeles|118684|
|         New York|118540|
|Dallas/Fort Worth|105024|
|          Houston|103008|
|           Denver| 97563|
|          Phoenix| 90281|
|    San Francisco| 84675|
|       Washington| 80950|
|        Las Vegas| 78357|
|        Charlotte| 76893|
|           Boston| 67155|
|           Newark| 64546|
|          Detroit| 62879|
|          Seattle| 60547|
|      Minneapolis| 60447|
|          Orlando| 59292|
|        Baltimore| 52005|
|     Philadelphia| 47659|
+-----------------+------+
only showing top 20 rows



In [33]:
flightsByOrigin = flights\
.join(airports, flights.OriginAirportID == airports.airport_id)\
.groupBy("city")\
.agg(F.count(F.lit(1)).alias("Count"))\
.orderBy("Count", ascending=False)

In [34]:
flightsByOrigin.limit(5).show()

+-----------------+------+
|             city| Count|
+-----------------+------+
|          Chicago|177845|
|          Atlanta|149970|
|      Los Angeles|118684|
|         New York|118540|
|Dallas/Fort Worth|105024|
+-----------------+------+



In [38]:
highestArrDelay = flights\
    .join(airports, flights.OriginAirportID == airports.airport_id)\
    .groupBy("city")\
    .agg(F.sum("ArrDelay").alias("SumArrDelay"))\
    .orderBy("SumArrDelay", ascending=False)

highestArrDelay.limit(10).show()


+-----------------+-----------+
|             city|SumArrDelay|
+-----------------+-----------+
|          Chicago|    2054858|
|           Denver|    1110827|
|Dallas/Fort Worth|    1062446|
|         New York|     748027|
|    San Francisco|     730889|
|      Los Angeles|     724121|
|          Atlanta|     698538|
|        Charlotte|     644707|
|        Las Vegas|     627858|
|          Houston|     627093|
+-----------------+-----------+



In [39]:
flights.rdd.getNumPartitions()

8

Now we go back to cleaning the data, checking for duplicates and Null values, we'll see how much it changes our current results.

In [41]:
unique_flights_withoutNA =  flights.dropDuplicates()\
.dropna(how="any", subset=["ArrDelay", "DepDelay"]).count()

print("Missing values (excluding dups) = ", total_flights - unique_flights_withoutNA)

Missing values (excluding dups) =  46233


In [42]:
data = flights.dropDuplicates().fillna(value=0, subset=["ArrDelay", "DepDelay"])

In [43]:
flights.rdd.getNumPartitions()

8

In [44]:
print("Number of rows in cleaned data set = ", data.count(), "Number of partitions = ", data.rdd.getNumPartitions())

Number of rows in cleaned data set =  2696983 Number of partitions =  9


In [45]:
flights.rdd.getNumPartitions()

8

Why is the number of partitions 9 ?

In [46]:
highestArrDelay = flights\
    .join(airports, flights.OriginAirportID == airports.airport_id)\
    .groupBy("city")\
    .agg(F.sum("ArrDelay").alias("SumArrDelay"))\
    .orderBy("SumArrDelay", ascending=False)

highestArrDelay.limit(10).show()

+-----------------+-----------+
|             city|SumArrDelay|
+-----------------+-----------+
|          Chicago|    2054858|
|           Denver|    1110827|
|Dallas/Fort Worth|    1062446|
|         New York|     748027|
|    San Francisco|     730889|
|      Los Angeles|     724121|
|          Atlanta|     698538|
|        Charlotte|     644707|
|        Las Vegas|     627858|
|          Houston|     627093|
+-----------------+-----------+



In [47]:
data.describe().show()

+-------+------------------+-----------------+-------+------------------+-----------------+------------------+------------------+
|summary|        DayofMonth|        DayOfWeek|Carrier|   OriginAirportID|    DestAirportID|          DepDelay|          ArrDelay|
+-------+------------------+-----------------+-------+------------------+-----------------+------------------+------------------+
|  count|           2696983|          2696983|2696983|           2696983|          2696983|           2696983|           2696983|
|   mean|15.798996508320593|3.900369412784582|   null|12742.459424846207|12742.85937657004|10.531134234068217|6.6679285705545785|
| stddev| 8.801267199135411|1.986458242170198|   null|1502.0359941370566|1501.993958981802| 36.06172819056568| 38.58386147358067|
|    min|                 1|                1|     9E|             10140|            10140|               -63|               -94|
|    max|                31|                7|     YV|             15376|            15376

In [48]:
data.corr("DepDelay", "ArrDelay")

0.9392630367706921

In [50]:
data.createOrReplaceTempView("flightData")
spark.sql(""" 
SELECT DayOfWeek, CAST(AVG(ArrDelay) as DECIMAL(6,2)) AS `Avg Delay(min)` 
FROM flightData 
GROUP BY DayOfWeek 
ORDER BY DayOfWeek 
""").show()

+---------+--------------+
|DayOfWeek|Avg Delay(min)|
+---------+--------------+
|        1|          7.08|
|        2|          4.39|
|        3|          7.23|
|        4|         10.78|
|        5|          8.71|
|        6|          2.14|
|        7|          5.25|
+---------+--------------+

